### Setting the environment

In [1]:
!pip install fastNLP==1.0.1
!pip install transformers
!pip install sparse==0.13.0
!pip install torch==1.11.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 -q
!pip install llvmlite==0.38.1
!pip install numba==0.55.2
!pip install adjustText

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


In [4]:
import torch
print(torch.__version__)

In [5]:
!nvcc --version

In [6]:
!pip install torch-scatter==2.0.9 --no-index -f https://data.pyg.org/whl/torch-{torch.__version__}.html 

In [7]:
import json
import warnings
import argparse

if 'p' in os.environ:
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['p']
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['MKL_THREADING_LAYER'] = 'GNU'
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import fastNLP
from fastNLP import cache_results, prepare_torch_dataloader
from fastNLP import Evaluator
from fastNLP import SortedSampler, BucketedBatchSampler

from model.model import CNNNer
from model.metrics import NERMetric
from preprocessing.ner_pipe import SpanNerPipe
from preprocessing.padder import Torch3DMatrixPadder
from model.metrics_utils import decode
from fastNLP import print as pr
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt 
from adjustText import adjust_text 


seed_value=42
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
os.environ['FASTNLP_GLOBAL_SEED'] = str(seed_value)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
def calculate(path):
    max_sent_len = 0
    total_lengths = 0
    total_ent_length = 0
    max_ent_length = 0
    num_ents = 0
    overlapped_ent_num = 0
    num_sent = 0
    with open(path, 'r') as f:
        for line in f:
            num_sent += 1
            data = json.loads(line.strip())
            max_sent_len = max(max_sent_len, len(data['tokens']))
            total_lengths += len(data['tokens'])
            flags = [0]*len(data['tokens'])
            for ent in data['entity_mentions']:
                num_ents += 1
                start, end = ent['start'], ent['end']
                total_ent_length += ent['end'] - ent['start']
                max_ent_length = max(max_ent_length, ent['end'] - ent['start'])
                for i in range(start, end):
                    flags[i] += 1
            for ent in data['entity_mentions']:
                start, end = ent['start'], ent['end']
                if any([flags[i]>1 for i in range(start, end)]):
                    overlapped_ent_num += 1

    pr(f"For {path}")
    pr("total sentence ", num_sent)
    pr("average sentence length ", total_lengths/num_sent)
    pr("max sentence length ", max_sent_len)

    pr('num_entities ', num_ents)
    pr('average entity length  ', total_ent_length/num_ents)
    pr('max entity length  ', max_ent_length)
    pr("Number of nested entity ", overlapped_ent_num)

    pr("Number of tokens ", total_lengths)
    pr()
    
def get_stats(directory): 
    for name in ['train', 'dev', 'test']:
        path = f'{directory}/{name}.jsonl'
        calculate(path)

In [9]:
get_stats('datasets/outputs/nerel_common_labels')

In [10]:
get_stats('datasets/outputs/nerel_bio_common_labels')

### Fetching and preprocessing the dataset

In [11]:
model_name = 'DeepPavlov/rubert-base-cased'


# @cache_results('caches/ner_caches.pkl', _refresh=False)
def get_data(dataset_name, model_name):
    paths = f'datasets/outputs/{dataset_name}'
    pipe = SpanNerPipe(model_name=model_name)
    dl = pipe.process_from_file(paths)
    return dl, pipe.matrix_segs

def densify(x):
    x = x.todense().astype(np.float32)
    return x
    
def preprocess_data(dataset_name, model_name):

    dl, matrix_segs = get_data(dataset_name, model_name)

    dl.apply_field(densify, field_name='matrix', new_field_name='matrix', progress_bar='Densify')

    print(dl)

    label2idx = getattr(dl, 'ner_vocab') if hasattr(dl, 'ner_vocab') else getattr(dl, 'label2idx')
    print(f"{len(label2idx)} labels: {label2idx}, matrix_segs:{matrix_segs}")

    dls = {}
    for name, ds in dl.iter_datasets():
        ds.set_pad('matrix', pad_fn=Torch3DMatrixPadder(pad_val=ds.collator.input_fields['matrix']['pad_val'],
                                                        num_class=matrix_segs['ent'],
                                                        batch_size=1))

        _dl = prepare_torch_dataloader(ds, batch_size=1, num_workers=0, pin_memory=True, shuffle=False)

        dls[name] = _dl
    
    return dls, label2idx, matrix_segs

In [12]:
dls_nerel, label2idx_nerel, matrix_segs_nerel = preprocess_data('nerel_common_labels', model_name)

In [13]:
dls_nerel_bio, label2idx_nerel_bio, matrix_segs_nerel_bio = preprocess_data('nerel_bio_common_labels', model_name)

### Experiments on data

In [14]:
def squeeze_batch_dim(input_list):
    for i in range(len(input_list)):
        input_list[i] = input_list[i][0]
    return input_list

def get_key_by_value(dictionary, value):
    return list(dictionary.keys())[list(dictionary.values()).index(value)]

def get_targets(dataloader):
    tokens_list = []
    targets_list = []
    for i, item in enumerate(dataloader):
        tokens = item['tokens']
        target = item['ent_target']
        tokens_list.append(tokens)
        targets_list.append(target)
    return tokens_list, targets_list


def split_by_label(targets, label2idx):
    tpl = {key:set() for key in range(len(label2idx))}

    for i in range(len(targets)):
        cur_target = set(map(tuple, targets[i]))
        for item in cur_target:
            tpl[item[2]].add(item)
    return tpl


def decode_targets(tokens_list, targets_list):
    tokens_with_label = [None]*len(targets_list)
    for i in range(len(targets_list)):
        target_tokens = []
        for span in targets_list[i]:
            start, end, label = span
            target_tokens.append([tokens_list[i][start:end+1], label])
        tokens_with_label[i] = target_tokens
    return tokens_with_label


def split_tokens_with_label_by_category(tokens_with_label, label2idx, tpl=None):
    if tpl is None:
        tpl = {key:set() for key in range(len(label2idx))}
        
    for i in range(len(tokens_with_label)):
        cur_tokens_with_label = tokens_with_label[i]
        for j in range(len(cur_tokens_with_label)):
            tpl[cur_tokens_with_label[j][1]].update(cur_tokens_with_label[j][0])
    return tpl


def build_vocab(dataloader, label2idx):
    tokens_with_label_dict = {key:set() for key in range(len(label2idx))}

    for name in ['train', 'test', 'dev']:
        tokens_list, targets_list = get_targets(dataloader[name])
        tokens_list = squeeze_batch_dim(tokens_list)
        targets_list = squeeze_batch_dim(targets_list)
        tokens_with_label = decode_targets(tokens_list, targets_list)
        tokens_with_label_dict = split_tokens_with_label_by_category(tokens_with_label, label2idx_nerel, tpl=tokens_with_label_dict)
        
    return tokens_with_label_dict

In [15]:
vocab_nerel = build_vocab(dls_nerel, label2idx_nerel)
vocab_nerel_bio = build_vocab(dls_nerel_bio, label2idx_nerel_bio)

In [16]:
overlap = {key:0 for key in range(len(label2idx_nerel))}
identical_tokens = {key:[] for key in range(len(label2idx_nerel))}

for i in range(len(label2idx_nerel)):
    intersection = vocab_nerel[i] & vocab_nerel_bio[i]
    overlap[i] = round(len(intersection) / (len(vocab_nerel[i] | vocab_nerel_bio[i]) + 1e-6) * 100, 2)
    identical_tokens[i] = list(intersection)

    
overlap_str = {}
identical_tokens_str = {}

for i in range(len(overlap)):
    overlap_str[get_key_by_value(label2idx_nerel, i)] = overlap[i]
    identical_tokens_str[get_key_by_value(label2idx_nerel, i)] = identical_tokens[i]

In [17]:
sim_data = {'Overlap': list(overlap_str.values())}

df_overlap = pd.DataFrame(data=sim_data, index=list(overlap_str.keys()))

df_overlap.index.name = 'Index'

df_overlap.transpose()

In [18]:
overlap_str

In [19]:
overlap_sorted = dict(sorted(overlap_str.items(), key=lambda item: item[1]))
sorted_keys = list(overlap_sorted.keys())

In [20]:
sorted_keys = ['PROF', 'CITY', 'DIS', 'FAC', 'PERS', 'PROD', 'ORG', 'PERC', 'S_O_P', 'CTRY', 'TIME', 'LOC', 'DATE', 'AGE', 'NUM', 'ORD']

In [21]:
f1_score_nerel_bio = {'AGE': 83.67,
 'CITY': 66.67,
 'CTRY': 79.39,
 'DATE': 78.97,
 'DIS': 87.96,
 'FAC': 0.0,
 'LOC': 78.26,
 'NUM': 83.14,
 'ORD': 87.5,
 'ORG': 66.67,
 'PERC': 81.82,
 'PERS': 26.03,
 'PROD': 11.11,
 'PROF': 90.2,
 'S_O_P': 87.5,
 'TIME': 66.67}

f1_scores = [f1_score_nerel_bio[key] for key in sorted_keys]

print(f1_scores)

In [22]:
corr = np.corrcoef(list(overlap_sorted.values()), f1_scores)
corr

In [23]:
x = list(overlap_sorted.values())
y = f1_scores

# plt.figure(figsize=(8,4))

fig, ax = plt.subplots(layout='constrained')
fig.set_figheight(4)
fig.set_figwidth(7)

ax.grid(color = 'blue', linestyle = '--', linewidth = 0.3)


ax.plot(x, y, marker='o', linewidth=1.5) 
# for i, key in enumerate(sorted_keys):
#     ax.annotate(key, (x[i], y[i]), xycoords='data')

texts = [ax.text(x[i], y[i], sorted_keys[i]) for i in range(len(x))]
adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=1))

ax.set_xticks(np.arange(0, 12, 1))
ax.set_yticks(np.arange(0, 100, 10))

ax.set_xlabel('% of matching tokens b/w NEREL-C and NEREL-BIO-C', fontsize=12) 
ax.set_ylabel('F1 score (NEREL-BIO-C)', fontsize=12) 
# ax.legend(fontsize=12)

plt.show() 

In [24]:
nerel_bio_f1 =   [89.80, 66.67, 92.21, 78.43, 93.09, 42.55, 21.05, 89.12, 79.41, 82.05, 92.50, 90.98, 40.00, 76.00, 70.00, 71.43]
nerel_bio_c_f1 = [83.67, 66.67, 79.39, 78.97, 87.96, 0.00, 78.26, 83.14, 87.50, 66.67, 81.82, 26.03, 11.11, 90.20, 87.50, 66.67]

subtracted = [element1 - element2 for (element1, element2) in zip(nerel_bio_c_f1, nerel_bio_f1)]

print(sum(subtracted)/len(subtracted))